# MNIST classifier from nnBuilder

This is a simple implementation of a MNIST classifier using the nnBuilder framework. The network is essentially identical to the one from the TensorFlow tutorial at https://www.tensorflow.org/tutorials/mnist/pros/. (Work in progress)

In [1]:
import numpy as np
import tensorflow as tf
import sys
sys.path.append('../nnBuilder')
from nnLayer import *
from nnInput import *
from nnTrainer import *
from nnHandler import *

In [2]:
data=Layer(type="MNIST",batch=128)
layers=[]
layers.append(dict(type="Convolution",pad="SAME",window=5,stride=1,size=32,relu=True))
layers.append(dict(type="Pool",pad="SAME",window=2,stride=2,pool_type="max"))
layers.append(dict(type="Convolution",pad="SAME",window=5,stride=1,size=64,relu=True))
layers.append(dict(type="Pool",pad="SAME",window=2,stride=2,pool_type="max"))
layers.append(dict(type="Relu",size=64))
layers.append(dict(type="Linear",size=10,in_features=["Dropout"]))
network=Layer(type="Network",x=data,layers=layers)
trainer=ClassifierTrainer(network=network,optimizer="adam",finish=True)
sess=SessManager(data,network,trainer)
sess.start()

Extracting /tmp/tensorflow/mnist/input_data\train-images-idx3-ubyte.gz
Extracting /tmp/tensorflow/mnist/input_data\train-labels-idx1-ubyte.gz
Extracting /tmp/tensorflow/mnist/input_data\t10k-images-idx3-ubyte.gz
Extracting /tmp/tensorflow/mnist/input_data\t10k-labels-idx1-ubyte.gz
Starting new session


In [3]:
trainer.train(20,keep_rate=1.)
trainer.eval_error()
for i in range(20):
    trainer.train(100,keep_rate=0.5)
    trainer.train(0,keep_rate=1.)
    trainer.eval_error() #Training error, reaches about 1%

Error rate: 0.48125
Error rate: 0.0835937
Error rate: 0.0492188
Error rate: 0.0453125
Error rate: 0.0414063
Error rate: 0.0296875
Error rate: 0.0351563
Error rate: 0.0328125
Error rate: 0.0125
Error rate: 0.0171875
Error rate: 0.01875
Error rate: 0.0109375
Error rate: 0.015625
Error rate: 0.0125
Error rate: 0.0125
Error rate: 0.0140625
Error rate: 0.0171875
Error rate: 0.0132813
Error rate: 0.0132813
Error rate: 0.0140625
Error rate: 0.0117188


In [4]:
network.save() #The full network definition

{'layers': [{'input_channels': None,
   'pad': 'SAME',
   'rand_scale': 0.1,
   'relu': True,
   'size': 32,
   'stride': 1,
   'type': 'Convolution',
   'window': 5},
  {'pad': 'SAME',
   'pool_type': 'max',
   'stride': 2,
   'type': 'Pool',
   'window': 2},
  {'input_channels': None,
   'pad': 'SAME',
   'rand_scale': 0.1,
   'relu': True,
   'size': 64,
   'stride': 1,
   'type': 'Convolution',
   'window': 5},
  {'pad': 'SAME',
   'pool_type': 'max',
   'stride': 2,
   'type': 'Pool',
   'window': 2},
  {'rand_scale': 0.1, 'size': 64, 'type': 'Relu'},
  {'in_features': [{'type': 'Dropout'}],
   'rand_scale': 0.1,
   'size': 10,
   'type': 'Linear'}],
 'type': 'Network'}